In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import mne
import os.path as op
import os
from os.path import join
import xml.etree.ElementTree as ET
import numpy as np
import pickle
import pyeeg
import csv
from pathlib import Path
import pandas as pd
import glob
import mne
import pywt
import scipy
import pysiology
import seaborn as sns
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Thank you for using Pysiology. If you use it in your work, please cite:
Gabrieli G., Azhari A., Esposito G. (2020) PySiology: A Python Package for Physiological Feature Extraction. In: Esposito A., Faundez-Zanuy M., Morabito F., Pasero E. (eds) Neural Approaches to Dynamics of Signal Exchanges. Smart Innovation, Systems and Technologies, vol 151. Springer, Singapore


## Feature Extraction

In [2]:
#Stats feature
#https://github.com/erickfmm/gsr-emotion-mh

def get_feature_vector(signal):
    features = []

    features.append(np.std(signal)) #data standard deviation

    features.append(np.max(signal)) #data maximum
#(sign_std,sign_max)
    ratios = []
    first_differences = []
    last_sample = 0
    first = True
    for sample in signal:
        if first:
            last_sample = sample
            first = False
            continue
        first_differences.append(np.absolute(sample - last_sample))
        ratios.append(np.absolute(sample / last_sample))
        last_sample = sample

    features.append(np.min(first_differences)) #minimum of first difference
    features.append(np.max(first_differences)) #maximum of first difference

    features.append(np.min(ratios)) #minimum of ratios

    features.append(np.max(ratios))#maximum of ratio

    features.append(np.median(first_differences)) #median of first difference
    second_step_ratios = []
    second_differences = []
    last_sample = 0
    first = True
    for sample in first_differences:
        if first:
            last_sample = sample
            first = False
            continue
        second_step_ratios.append(np.absolute(sample / last_sample))
        second_differences.append(np.absolute(sample - last_sample))
        last_sample = sample

    features.append(np.min(second_differences)) #minimum of second difference
    features.append(np.max(second_differences))#maximum of second difference

    features.append(np.min(second_step_ratios))#minimum of second step ratios

    features.append(np.max(second_step_ratios))#maximum of second step ratios

    #############################################################################3
    #third step
    features.append(np.median(second_differences)) #median of second difference
    third_step_ratios = []
    third_differences = []
    last_sample = 0
    first = True
    for sample in second_differences:
        if first:
            last_sample = sample
            first = False
            continue
        third_differences.append(np.absolute(sample - last_sample))
        third_step_ratios.append(np.absolute(sample / last_sample))
        last_sample = sample

    features.append(np.min(third_differences)) #third difference minimum
    features.append(np.max(third_differences))#third difference maximum
    #(td_min,td_max,tsr_min,tsr_max,td_max_min)
    return features

In [3]:
def freq_feature(amplitude,signal):
    avgSTD1 = np.std(amplitude) #standard of amplitude
    maxSCR = np.max(amplitude) #maximum of amplitude
    minSCR = np.min(amplitude)# minimum of amplitude
    freqSCR = len(amplitude)/len(signal) # frequency of SCR
    
    return avgSTD1,maxSCR,minSCR,freqSCR
    
    
def fft_feature(signal):    
    fft = np.fft.fft(signal)
    med_fft = np.absolute(np.median(fft)) #median of fft decomposition
    
    return med_fft

In [39]:
files = os.listdir('GSR/GSR_npy/')
#sign_std, sign_max, fd_min,fd_max,r_min,r_max,fd_med,sd_min,sd_max, ssr_min, ssr_max,sd_med,td_min,td_max,fft_med


count = 0
avgST1 =[]
maxS=[]
minS=[]
freqS=[]
med_f=[]
sign_st=[]
sign_ma=[]
fd_mi=[]
fd_ma=[]
fd_ratio_mi=[]
fd_ratio_ma=[]
fd_me=[]
sd_mi=[]
sd_ma=[]
ssr_mi=[]
ssr_ma=[]
sd_me=[]
td_mi=[]
td_ma=[]
emotion_of_file =[]
sr = 256
# np.char.decode(unicode_np_array.astype(np.bytes_), 'UTF-8')
for name in files:
    #print(name)
    signal = np.load('GSR/GSR_npy/'+name)
    new =np.array(signal).flatten()
#     new.ndim
    numpy_list = new.tolist()
    #results =pysiology.electrodermalactivity.analyzeGSR(n,sr,lowpass=1,highpass=5)
    res = pysiology.electrodermalactivity.findPeakOnsetAndOffset(numpy_list,onset = 0.01, offset =0)
    #len(res)
# for i in res:
    for i in range(len(res)):
        fe = pysiology.electrodermalactivity.GSRSCRFeaturesExtraction(new, sr , res[i])
        #print(fe)
    #results = pysiology.electrodermalactivity.analyzeGSR(new,samplerate=256,phasic_seconds=5)
        amplitude_gsr = []
        keys = fe.keys()
        for i in keys:
            amplitude_gsr.append(fe["amplitude"]),


        avg_STD1,max_SCR,min_SCR,freq_SCR = freq_feature(amplitude_gsr,new)
        med_fft = fft_feature(new)
        #stats_feature = get_feature_vector(new)
        sign_std, sign_max, fd_min,fd_max,ratio_min,ratio_max,fd_med,sd_min,sd_max, ssr_min, ssr_max,sd_med,td_min,td_max = get_feature_vector(new)
        #print(name)
    avgST1.append(avg_STD1)
    maxS.append(max_SCR)
    minS.append(min_SCR)
    freqS.append(freq_SCR)
    med_f.append(med_fft)
    sign_st.append(sign_std)
    sign_ma.append(sign_max)
    fd_mi.append(fd_min)
    fd_ma.append(fd_max)
    fd_ratio_mi.append(ratio_min)
    fd_ratio_ma.append(ratio_max)
    fd_me.append(fd_med)
    sd_mi.append(sd_min)
    sd_ma.append(sd_max)
    ssr_mi.append(ssr_min)
    ssr_ma.append(ssr_max)
    sd_me.append(sd_med)
    td_mi.append(td_min)
    td_ma.append(td_max)
    name_of_file = name.split('_')
    print(name_of_file)
    emotion_of_file.append(name_of_file[-2])
    count+=1
        
print(count)

    

['Part', '10', 'S', 'Trial10', 'emotion', '0', 'filtered.npy']
['Part', '10', 'S', 'Trial11', 'emotion', '5', 'filtered.npy']
['Part', '10', 'S', 'Trial12', 'emotion', '4', 'filtered.npy']
['Part', '10', 'S', 'Trial13', 'emotion', '11', 'filtered.npy']
['Part', '10', 'S', 'Trial14', 'emotion', '11', 'filtered.npy']
['Part', '10', 'S', 'Trial15', 'emotion', '0', 'filtered.npy']
['Part', '10', 'S', 'Trial16', 'emotion', '0', 'filtered.npy']
['Part', '10', 'S', 'Trial17', 'emotion', '0', 'filtered.npy']
['Part', '10', 'S', 'Trial18', 'emotion', '11', 'filtered.npy']
['Part', '10', 'S', 'Trial19', 'emotion', '0', 'filtered.npy']
['Part', '10', 'S', 'Trial1', 'emotion', '2', 'filtered.npy']
['Part', '10', 'S', 'Trial20', 'emotion', '6', 'filtered.npy']
['Part', '10', 'S', 'Trial2', 'emotion', '4', 'filtered.npy']
['Part', '10', 'S', 'Trial3', 'emotion', '3', 'filtered.npy']
['Part', '10', 'S', 'Trial4', 'emotion', '12', 'filtered.npy']
['Part', '10', 'S', 'Trial5', 'emotion', '0', 'filtered

['Part', '18', 'S', 'Trial5', 'emotion', '5', 'filtered.npy']
['Part', '18', 'S', 'Trial6', 'emotion', '0', 'filtered.npy']
['Part', '18', 'S', 'Trial7', 'emotion', '11', 'filtered.npy']
['Part', '18', 'S', 'Trial8', 'emotion', '2', 'filtered.npy']
['Part', '18', 'S', 'Trial9', 'emotion', '0', 'filtered.npy']
['Part', '19', 'S', 'Trial10', 'emotion', '3', 'filtered.npy']
['Part', '19', 'S', 'Trial11', 'emotion', '0', 'filtered.npy']
['Part', '19', 'S', 'Trial12', 'emotion', '6', 'filtered.npy']
['Part', '19', 'S', 'Trial13', 'emotion', '0', 'filtered.npy']
['Part', '19', 'S', 'Trial14', 'emotion', '11', 'filtered.npy']
['Part', '19', 'S', 'Trial15', 'emotion', '4', 'filtered.npy']
['Part', '19', 'S', 'Trial16', 'emotion', '4', 'filtered.npy']
['Part', '19', 'S', 'Trial17', 'emotion', '0', 'filtered.npy']
['Part', '19', 'S', 'Trial18', 'emotion', '12', 'filtered.npy']
['Part', '19', 'S', 'Trial19', 'emotion', '12', 'filtered.npy']
['Part', '19', 'S', 'Trial1', 'emotion', '4', 'filtered.

['Part', '24', 'S', 'Trial16', 'emotion', '4', 'filtered.npy']
['Part', '24', 'S', 'Trial17', 'emotion', '5', 'filtered.npy']
['Part', '24', 'S', 'Trial18', 'emotion', '11', 'filtered.npy']
['Part', '24', 'S', 'Trial19', 'emotion', '11', 'filtered.npy']
['Part', '24', 'S', 'Trial1', 'emotion', '12', 'filtered.npy']
['Part', '24', 'S', 'Trial20', 'emotion', '0', 'filtered.npy']
['Part', '24', 'S', 'Trial2', 'emotion', '2', 'filtered.npy']
['Part', '24', 'S', 'Trial3', 'emotion', '11', 'filtered.npy']
['Part', '24', 'S', 'Trial4', 'emotion', '0', 'filtered.npy']
['Part', '24', 'S', 'Trial5', 'emotion', '2', 'filtered.npy']
['Part', '24', 'S', 'Trial6', 'emotion', '4', 'filtered.npy']
['Part', '24', 'S', 'Trial7', 'emotion', '5', 'filtered.npy']
['Part', '24', 'S', 'Trial8', 'emotion', '3', 'filtered.npy']
['Part', '24', 'S', 'Trial9', 'emotion', '0', 'filtered.npy']
['Part', '25', 'S', 'Trial10', 'emotion', '0', 'filtered.npy']
['Part', '25', 'S', 'Trial11', 'emotion', '11', 'filtered.np

['Part', '30', 'S', 'Trial7', 'emotion', '4', 'filtered.npy']
['Part', '30', 'S', 'Trial8', 'emotion', '0', 'filtered.npy']
['Part', '30', 'S', 'Trial9', 'emotion', '11', 'filtered.npy']
['Part', '3', 'S', 'Trial10', 'emotion', '5', 'filtered.npy']
['Part', '3', 'S', 'Trial11', 'emotion', '0', 'filtered.npy']
['Part', '3', 'S', 'Trial12', 'emotion', '0', 'filtered.npy']
['Part', '3', 'S', 'Trial13', 'emotion', '11', 'filtered.npy']
['Part', '3', 'S', 'Trial14', 'emotion', '4', 'filtered.npy']
['Part', '3', 'S', 'Trial15', 'emotion', '6', 'filtered.npy']
['Part', '3', 'S', 'Trial16', 'emotion', '0', 'filtered.npy']
['Part', '3', 'S', 'Trial17', 'emotion', '12', 'filtered.npy']
['Part', '3', 'S', 'Trial1', 'emotion', '5', 'filtered.npy']
['Part', '3', 'S', 'Trial2', 'emotion', '2', 'filtered.npy']
['Part', '3', 'S', 'Trial3', 'emotion', '12', 'filtered.npy']
['Part', '3', 'S', 'Trial4', 'emotion', '4', 'filtered.npy']
['Part', '3', 'S', 'Trial5', 'emotion', '11', 'filtered.npy']
['Part',

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in double_scalars
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars


['Part', '4', 'S', 'Trial1', 'emotion', '11', 'filtered.npy']
['Part', '4', 'S', 'Trial20', 'emotion', '5', 'filtered.npy']
['Part', '4', 'S', 'Trial2', 'emotion', '4', 'filtered.npy']
['Part', '4', 'S', 'Trial3', 'emotion', '1', 'filtered.npy']
['Part', '4', 'S', 'Trial4', 'emotion', '2', 'filtered.npy']
['Part', '4', 'S', 'Trial5', 'emotion', '12', 'filtered.npy']
['Part', '4', 'S', 'Trial6', 'emotion', '6', 'filtered.npy']
['Part', '4', 'S', 'Trial7', 'emotion', '4', 'filtered.npy']
['Part', '4', 'S', 'Trial8', 'emotion', '0', 'filtered.npy']
['Part', '4', 'S', 'Trial9', 'emotion', '2', 'filtered.npy']
['Part', '5', 'S', 'Trial10', 'emotion', '11', 'filtered.npy']
['Part', '5', 'S', 'Trial11', 'emotion', '5', 'filtered.npy']
['Part', '5', 'S', 'Trial12', 'emotion', '4', 'filtered.npy']
['Part', '5', 'S', 'Trial13', 'emotion', '11', 'filtered.npy']
['Part', '5', 'S', 'Trial14', 'emotion', '6', 'filtered.npy']
['Part', '5', 'S', 'Trial15', 'emotion', '6', 'filtered.npy']
['Part', '5',

In [176]:
# Creating Dataframes
import pandas as pd

avg_ST1 =pd.DataFrame(avgST1)
max_S=pd.DataFrame(maxS)
min_S=pd.DataFrame(minS)
freq_S=pd.DataFrame(freqS)
med__f=pd.DataFrame(med_f)
sign__st=pd.DataFrame(sign_st)
sign__ma=pd.DataFrame(sign_ma)
fd__mi=pd.DataFrame(fd_mi)
fd__ma=pd.DataFrame(fd_ma)
fd__ratio_mi=pd.DataFrame(fd_ratio_mi)
fd__ratio_ma=pd.DataFrame(fd_ratio_ma)
fd__me=pd.DataFrame(fd_me)
sd__mi=pd.DataFrame(sd_mi)
sd__ma=pd.DataFrame(sd_ma)
ssr__mi=pd.DataFrame(ssr_mi)
ssr__ma=pd.DataFrame(ssr_ma)
sd__me=pd.DataFrame(sd_me)
td__mi=pd.DataFrame(td_mi)
td__ma=pd.DataFrame(td_ma)
emotion =pd.DataFrame(emotion_of_file)


df = pd.concat([avg_ST1,max_S,min_S,freq_S,med__f,sign__st,sign__ma,fd__mi,fd__ma,fd__ratio_mi,fd__ratio_ma,fd__me,sd__mi,sd__ma,ssr__mi,ssr__ma,sd__me,td__mi,td__ma,emotion],axis = 1)
df.columns=["avg_ST1","max_S","min_S","freq_S","med__f","sign__st","sign__ma","fd__mi","fd__ma","fd__ratio_mi","fd__ratio_ma","fd__me","sd__mi","sd__ma","ssr__mi","ssr__ma","sd__me","td__mi","td__ma","emotion"]
df.head()

# std__f,max__f,min__f,sign__me,sign__mea,sign__st,sign__max_mi,fd__ratio_mi,fd__ratio_ma,fd__me,fd__mea,fd__st,fd__max_mi,sd__me,sd__mea,sd__st,sd__max_mi


,avg_ST1,max_S,min_S,freq_S,med__f,sign__st,sign__ma,fd__mi,fd__ma,fd__ratio_mi,fd__ratio_ma,fd__me,sd__mi,sd__ma,ssr__mi,ssr__ma,sd__me,td__mi,td__ma,emotion
0,0.0,8.616287,8.616287,0.000110,53695.666515,5886.790600,85258.260199,0.000021,3198.716731,0.055712,14.473580,0.228427,6.182381e-07,172.819815,0.001292,657.834581,0.019048,3.249734e-07,85.799251,0
1,0.0,0.472221,0.472221,0.000110,29339.269398,4562.762428,60372.217311,0.000030,2188.825007,0.001800,508.973981,0.430199,8.496136e-08,118.159957,0.002814,325.445783,0.007003,3.370224e-08,65.475317,5
2,0.0,5.946306,5.946306,0.000124,26885.212908,4438.094911,58850.048823,0.000028,2207.485274,0.009837,90.892813,0.928159,2.641100e-07,119.230486,0.003042,408.357905,0.005387,1.175067e-08,60.342085,4
3,0.0,0.391753,0.391753,0.000124,27754.539626,4505.662353,59528.260066,0.000210,2230.984332,0.006245,144.258942,0.939944,2.375455e-07,120.466631,0.002147,422.277022,0.005778,8.971256e-09,70.249722,11
4,0.0,0.399538,0.399538,0.000163,28405.299804,5323.775068,60506.214314,0.000134,2270.317912,0.015385,57.427417,0.676617,5.936454e-08,122.680038,0.005971,170.347878,0.004775,6.955815e-09,61.114930,11


In [177]:
#Removing Nan value
df_test = df.replace([np.inf, -np.inf], np.nan)
df_test.head()
df_test.shape

(527, 20)

In [178]:
df_test.isnull().sum()

avg_ST1         0
max_S           0
min_S           0
freq_S          0
med__f          0
sign__st        0
sign__ma        0
fd__mi          0
fd__ma          0
fd__ratio_mi    0
fd__ratio_ma    0
fd__me          0
sd__mi          0
sd__ma          0
ssr__mi         0
ssr__ma         1
sd__me          0
td__mi          0
td__ma          0
emotion         0
dtype: int64

In [179]:
nan_rows = df_test[df_test["ssr__ma"].isnull()]
nan_rows

,avg_ST1,max_S,min_S,freq_S,med__f,sign__st,sign__ma,fd__mi,fd__ma,fd__ratio_mi,fd__ratio_ma,fd__me,sd__mi,sd__ma,ssr__mi,ssr__ma,sd__me,td__mi,td__ma,emotion
423,0.0,0.49646,0.49646,0.00017,0.168916,19221.055273,78171.995046,0.0,2824.222953,0.041097,23.690214,0.709827,0.0,152.523443,0.0,NaN,0.016648,0.0,82.950651,11


In [180]:
#Particular row dropped
df_test = df_test.drop([423])

In [181]:
nan_rows = df_test[df_test["ssr__ma"].isnull()]
nan_rows

,avg_ST1,max_S,min_S,freq_S,med__f,sign__st,sign__ma,fd__mi,fd__ma,fd__ratio_mi,fd__ratio_ma,fd__me,sd__mi,sd__ma,ssr__mi,ssr__ma,sd__me,td__mi,td__ma,emotion


In [182]:
df_test.isnull().sum()

avg_ST1         0
max_S           0
min_S           0
freq_S          0
med__f          0
sign__st        0
sign__ma        0
fd__mi          0
fd__ma          0
fd__ratio_mi    0
fd__ratio_ma    0
fd__me          0
sd__mi          0
sd__ma          0
ssr__mi         0
ssr__ma         0
sd__me          0
td__mi          0
td__ma          0
emotion         0
dtype: int64

In [183]:
#Only taking emotion 0,2,3,4,5,11
df_updated = df_test[(df_test.emotion != '1') & (df_test.emotion != '12' ) & (df_test.emotion != '6')]
df_updated.shape

(453, 20)

In [184]:
df_emotion_final = df_updated['emotion']
df_emotion_final.shape

(453,)

In [185]:
#Using Min Max scale to normalize data
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler()
df_updated.drop('emotion', axis = 1, inplace = True)
df_updated[df_updated.columns] = ms.fit_transform(df_updated[df_updated.columns])


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:3498: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pyd

In [186]:
#Concatenating both dataframes to prepare it for split.
df_gsr =pd.concat([df_updated, df_emotion_final], axis=1)
df_gsr.shape

(453, 20)

In [187]:
X_1 = df_gsr[["avg_ST1","max_S","min_S","freq_S","med__f","sign__st","sign__ma","fd__mi","fd__ma","fd__ratio_mi","fd__ratio_ma","fd__me","sd__mi","sd__ma","ssr__mi","ssr__ma","sd__me","td__mi","td__ma"]]   
y = df_gsr["emotion"]


## Model Train

In [188]:
#Splitting dataset Test Size 25%
X_1_train, X_1_test, Y_1_train,Y_1_test = train_test_split(X_1, y, test_size = 0.25, random_state = 0)

## SVC

In [189]:
classifier_1 = SVC(kernel = 'rbf', random_state = 20)
classifier_1.fit(X_1_train, Y_1_train)
Y_1_pred = classifier_1.predict(X_1_test)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [190]:
print(classification_report(Y_1_test, Y_1_pred))

              precision    recall  f1-score   support

           0       0.24      1.00      0.38        27
          11       0.00      0.00      0.00        24
           2       0.00      0.00      0.00        17
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        22
           5       0.00      0.00      0.00        18

   micro avg       0.24      0.24      0.24       114
   macro avg       0.04      0.17      0.06       114
weighted avg       0.06      0.24      0.09       114



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [191]:
print("Accuracy score")
print(accuracy_score(Y_1_test, Y_1_pred)*100)

Accuracy score
23.684210526315788


## Random Forest

In [192]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=21,stratify=y)
X_train, X_test, y_train,y_test = train_test_split(X_1, y, test_size = 0.25,random_state=0)

param_rf = {'n_estimators': [10,20,30,40,50],
            'max_features': ['sqrt'],
            'max_depth': [80,120,160,200],
            'min_samples_split': [3,5,7],
            'min_samples_leaf': [1,2],
            'bootstrap': [False]}

rf = RandomForestClassifier()
rf_cv = RandomizedSearchCV(rf, param_rf, cv=5,n_iter=25)
rf_cv.fit(X_train,y_train)
print("Accuracy score")
x = (rf_cv.score(X_test,y_test))*100
print(x)

Accuracy score
35.08771929824561


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [193]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_test_rf = rf_cv.predict(X_test)
y_pred_train_rf = rf_cv.predict(X_train)
y_pred_prob_rf = rf_cv.predict_proba(X_test)[:,1]
cr_rf = classification_report(y_pred_test_rf, y_test)
print(cr_rf)

              precision    recall  f1-score   support

           0       0.44      0.35      0.39        34
          11       0.46      0.46      0.46        24
           2       0.12      0.33      0.17         6
           3       0.17      0.10      0.12        10
           4       0.23      0.21      0.22        24
           5       0.50      0.56      0.53        16

   micro avg       0.35      0.35      0.35       114
   macro avg       0.32      0.34      0.32       114
weighted avg       0.37      0.35      0.35       114



## KNN

In [194]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

clf = KNeighborsClassifier(n_neighbors=8)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)
cm = confusion_matrix(y_test, y_predict)
print(cm)
print("Accuracy score")
print(accuracy_score(y_test, y_predict)*100)

[[10  5  2  0  4  6]
 [ 5 10  4  0  5  0]
 [ 3  3  8  0  2  1]
 [ 4  0  0  2  0  0]
 [11  4  0  1  5  1]
 [ 7  4  1  0  4  2]]
Accuracy score
32.45614035087719
